<a href="https://colab.research.google.com/github/fibleep/adam-mickiewicz-ai/blob/main/colbert_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas ragatouille

  Using cached faiss_cpu-1.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)


In [2]:
!git clone https://github.com/fibleep/adam-mickiewicz-ai.git

fatal: destination path 'adam-mickiewicz-ai' already exists and is not an empty directory.


In [3]:
import pandas as pd

df = pd.read_csv("/content/adam-mickiewicz-ai/conversation_results.csv").drop("Index", axis=1)
df

,Question,Answer
0,Co to za piękny kurhanek?,"To cudzy człowiek, zielonej rozłogi wianek."
1,"Czy uciekasz ode mnie, czy lękasz się nudy?","Nie uciekam, lecz lękam się monotonii."
2,Czy Maryla przespała ranek? Gdzie się kryje? N...,"Maryla nie przespała ranek, nie żyje więcej. K..."
3,"Czym jest czara złota, która tu stoi?","Czara złota jest symbolem życia, które przemij..."
4,Jak nazywasz się i czym się zajmujesz?,"Nazywam się Adam Mickiewicz, jestem polskim po..."
...,...,...
1970,"Kim była ta dziewczyna, co w parku się ukazała?","Litwinką była, nieznaną, tajemniczą, jak sen."
1971,"Co czułeś, gdy patrzyłeś na pożar i myślałeś o...",Patrzyłem. Różne myśli snuły się po głowie.\nZ...
1972,"Bo fałsz, żebym był w jakiej z Moskalami zmowi...","Niechaj fałsz z Moskalami mnie nie zwiąże,\nBo..."
1973,Jakie uczucia odczuwasz wobec zguby i jak prób...,"Odtąd nielube życie, wyrzuty późne, zguby żału..."


In [4]:
data = [(row[0],row[1]) for row in df.values]
corpus = pd.read_csv("/content/adam-mickiewicz-ai/data_extraction/books.csv")
my_full_corpus = corpus.Text.values.tolist()

In [5]:
# !pip install spacy
# !python -m spacy download pl_core_news_lg

In [6]:
# Taking way too long

# import spacy
# nlp = spacy.load('pl_core_news_lg')
# def remove_stopwords_and_newlines(docs):
#     cleaned_docs = []
#     for doc in docs:
#         tokens = [token.text for token in nlp(doc) if not token.is_stop and '\n' not in token.text]
#         cleaned_doc = ' '.join(tokens)
#         cleaned_docs.append(cleaned_doc)
#     return cleaned_docs

# my_full_corpus = [remove_stopwords_and_newlines(doc) for doc in my_full_corpus]


In [8]:
from ragatouille import RAGTrainer
from ragatouille.utils import get_wikipedia_page


trainer = RAGTrainer(model_name = "mickiewicz-colbert",
        pretrained_model_name = "colbert-ir/colbertv2.0") # In this example, we run fine-tuning

# This step handles all the data processing, check the examples for more details!
trainer.prepare_training_data(raw_data=data,
                                data_out_path="./data/",
                                all_documents=my_full_corpus)

trainer.train(batch_size=32) # Train with the default hyperparams

Loading Hard Negative SimpleMiner dense embedding model BAAI/bge-small-en-v1.5...
Building hard negative index for 2124 documents...
All documents embedded, now adding to index...
save_index set to False, skipping saving hard negative index
Hard negative index generated
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
mining
m

In [11]:
!pip uninstall --y faiss-cpu & pip install faiss-gpu

In [12]:
from ragatouille import RAGPretrainedModel
from ragatouille.utils import get_wikipedia_page
from ragatouille.data import CorpusProcessor


RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
processor = CorpusProcessor()
my_documents = processor.process_corpus(my_full_corpus)
index_path = RAG.index(index_name="my_index", collection=my_full_corpus)

________________________________________________________________________________
WARNING! You have a GPU available, but only `faiss-cpu` is currently installed.
 This means that indexing will be slow. To make use of your GPU.
Please install `faiss-gpu` by running:
pip uninstall --y faiss-cpu & pip install faiss-gpu
 ________________________________________________________________________________
Will continue with CPU indexing in 5 seconds...


[Jan 20, 21:34:36] #> Note: Output directory .ragatouille/colbert/indexes/my_index already exists


[Jan 20, 21:34:36] #> Will delete 1 files already at .ragatouille/colbert/indexes/my_index in 20 seconds...
[Jan 20, 21:34:57] [0] 		 #> Encoding 2845 passages..
[Jan 20, 21:35:12] [0] 		 avg_doclen_est = 201.29981994628906 	 len(local_sample) = 2,845
[Jan 20, 21:35:13] [0] 		 Creating 8,192 partitions.
[Jan 20, 21:35:13] [0] 		 *Estimated* 572,697 embeddings.
[Jan 20, 21:35:13] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/my

0it [00:00, ?it/s]

[Jan 20, 21:42:55] [0] 		 #> Encoding 2845 passages..


1it [00:13, 13.90s/it]
100%|██████████| 1/1 [00:00<00:00, 424.44it/s]

[Jan 20, 21:43:09] #> Optimizing IVF to store map from centroids to list of pids..
[Jan 20, 21:43:09] #> Building the emb2pid mapping..
[Jan 20, 21:43:09] len(emb2pid) = 572698



100%|██████████| 8192/8192 [00:00<00:00, 71393.15it/s]

[Jan 20, 21:43:09] #> Saved optimized IVF to .ragatouille/colbert/indexes/my_index/ivf.pid.pt


Done indexing!


In [14]:
RAG.search(["Kim jest Konrad?","Na czym polegają dziady?"], k=2)

2it [00:00, 50.88it/s]


[[{'content': 'Ale miał wadę — bo któż jest bez wady?\nKonrad światowej nie lubił pustoty,\nKonrad pijanej nie dzielił biesiady.\nWszakże, zamknięty w samotnym pokoju,\nGdy go dręczyły nudy lub zgryzoty,\nSzukał pociechy w gorącym napoju;\nI wtenczas zdał się wdziewać postać nową,\nWtenczas twarz jego, bladą i surową,\nJakiś rumieniec chorowity krasił:\nI wielkie, niegdyś błękitne źrenice,\nKtóre czas nieco skaził i przygasił,\nCiskały dawnych ogniów błyskawice.',
   'score': 17.90625,
   'rank': 1},
  {'content': 'KAPRAL\n\nJak on okropnie patrzy, — to jest pieśń szatańska.\n\n/ przestają śpiewać /\n\n\nKONRAD\n\n/ z towarzyszeniem fletu /\n\nWznoszę się! lecę! tam, na szczyt opoki —\n  Już nad plemieniem człowieczem,\n    Między proroki.\nStąd ja przyszłości brudne obłoki\nRozcinam moją źrenicą jak mieczem;\nRękami jak wichrami mgły jej rozdzieram —\nJuż widno — jasno — z góry na ludy spozieram —\nTam księga sybilińska przyszłych losów świata —\n    Tam, na dole!\nPatrz, patrz, przys